In [1]:
import psycopg2

conn = psycopg2.connect(
        dbname="financialAnalysis",
        user="postgres",
        password="rimthabet2000",
        host="localhost",
        port="5432"
    )
cursor = conn.cursor()

In [3]:
from pip._vendor import requests

url = "http://stbf.gdf.nacloud.tn:8027/gdfApi/actualite/all"
reponse = requests.get(url)
actualites = reponse.json()
print(actualites)


[{'id': 164190, 'langue': 'FR', 'titre': 'Euro-Cycles a publié cette semaine ses résultats relatifs à l’exercice 2016. Ces derniers sont de bo...', 'resume': 'Euro-Cycles a publié cette semaine ses résultats relatifs à l’exercice 2016. Ces derniers sont de bo...', 'description': 'Euro-Cycles a publié cette semaine ses résultats relatifs à l’exercice 2016. Ces derniers sont de bon augure. Le fabricant de vélos a réalisé une progression de 7% à 80 MDT dans ses revenus. \n\n', 'source': None, 'image': None, 'sourceUrl': None, 'datePublication': '2017-04-18', 'sticky': None, 'visualiser': True, 'tags': '#weekly #information_société', 'emetteur': {'id': 168, 'code': 'TN0007570013', 'libelle': 'EURO-CYCLES', 'identifiantUnique': None, 'registreCommerce': 'B1119851997', 'formeJuridique': 'Société Anonyme', 'telephone': '73342036', 'fax': '73342039', 'email': 'eurocycles@gnet.tn', 'logo': 'TN0007570013.jpg', 'siteWeb': 'www.euro-cycles.com.tn', 'dateLancement': '1993-11-29T23:00:00.000+00:00',

In [4]:
import json

# Sauvegarder dans un fichier
with open("C:/Users/hp/Desktop/sentimentAnalaysis/actualiteStbf.json", "w", encoding="utf-8") as file:
   json.dump(actualites, file, ensure_ascii=False, indent=4)
print("Données sauvegardées dans actualiteStbf.json")

Données sauvegardées dans actualiteStbf.json


In [5]:
from datetime import datetime

def parse_date(date_str):
    try:
        # Remplacer 'T' par un espace et supprimer les millisecondes et le fuseau horaire
        parsed_date = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        # Convertir au format 'YYYY-MM-DD HH:MI:SS'
        return parsed_date.strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        print(f"Date invalide : {date_str}")
        return None


In [6]:
import logging

# Configurer le logging
logging.basicConfig(level=logging.ERROR, format="%(asctime)s - %(levelname)s - %(message)s")

resultats = []
for index, element in enumerate(actualites):
    try:
        date_publication = element.get("datePublication")
        description = element.get("description")
        emetteur = element.get("emetteur", {})

        if emetteur is None: 
            logging.warning(f"L'attribut 'emetteur' est None pour l'élément à l'index {index}. Skipping.")
            continue

        code_isin = emetteur.get("code")
        
        if any(v is None for v in [date_publication, description, code_isin]):
            logging.warning(f"Valeurs manquantes détectées pour l'élément à l'index {index}. Skipping.")
            continue  

       
        resultats.append((date_publication, code_isin, description))

    except Exception as e:
        logging.error(f"Erreur à l'index {index} pour l'élément {element}: {e}")
        continue  


for index, resultat in enumerate(resultats):
    print(f"Élément {index + 1}: {resultat}")


Élément 1: ('2017-04-18', 'TN0007570013', 'Euro-Cycles a publié cette semaine ses résultats relatifs à l’exercice 2016. Ces derniers sont de bon augure. Le fabricant de vélos a réalisé une progression de 7% à 80 MDT dans ses revenus. \n\n')
Élément 2: ('2017-04-20', 'TN0007410012', "Le groupe Ennakl a publié ses états financiers consolidés au 31 décembre 2016. Ces derniers dévoilent une amélioration de l'ordre de13% à 395 MDT dans le chiffre d'affaires mais une régression de 5% à 35 MDT du résultat net.")
Élément 3: ('2017-04-25', 'TN0007570013', 'Accord sur distribution de dividende à raison de 1,4 DT par action, qui sera mis en paiement à compter du Mercredi 10 Mai 2017.')
Élément 4: ('2017-04-25', 'TN0006780019', 'Le chiffre d’affaires total du premier trimestre 2017 s’établit à 21,1MDT, enregistrant ainsi une hausse de 13,6% par rapport à la même période de l’année 2016, arrêté à 18,6MDT. Il est à rappeler que dans le cadre de sa politique de développement à l’international, la soc

In [7]:
import pandas as pd

# Convertir en DataFrame
df_resultats = pd.DataFrame(resultats, columns=["date_publication", "code_isin", "description"])

# Sauvegarder en CSV
df_resultats.to_csv("Stbf_data.csv", index=False, encoding="utf-8")

print("Les résultats ont été enregistrés dans 'resultats.csv'.")


Les résultats ont été enregistrés dans 'resultats.csv'.


In [9]:
insert_query = """
    INSERT INTO allactualite (date_publication, code_isin,description)
    VALUES (%s, %s, %s)
"""

try:
    cursor.executemany(insert_query, resultats)
    conn.commit()
    print("Toutes les données valides ont été insérées avec succès.")
except Exception as e:
    print(f"Erreur lors de l'insertion : {e}")


Toutes les données valides ont été insérées avec succès.
